## Question Answering Demo Application

In [1]:
import pandas
import spacy
import warnings
from IPython.display import display,HTML
from aips import *
import aips.indexer

engine = get_engine()
outdoors_collection = aips.indexer.build_collection(engine, "outdoors")

warnings.filterwarnings('ignore') #Some operations warn inside a loop, we"ll only need to see the first warning

/opt/conda/lib/python3.10/site-packages/thinc/compat.py:36: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  hasattr(torch, "has_mps")
/opt/conda/lib/python3.10/site-packages/thinc/compat.py:37: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  and torch.has_mps  # type: ignore[attr-defined]


In [2]:
nlp = spacy.load("en_core_web_sm")
nlp.remove_pipe("ner")
def get_query_from_question(question):
    words = [token.text for token in nlp(question)
             if not (token.lex.is_stop or token.lex.is_punct)]
    return " ".join(words)

## Listing 14.15

In [3]:
def display_contexts(contexts):
    display(HTML(pandas.DataFrame(contexts[0:10]).to_html(index=False)))

In [4]:
def retriever(question):
    contexts = {"id": [], "question": [], "context": [], "url": []}
    query = get_query_from_question(question)
    request = {"query": query,
               "query_fields": ["body"],
               "return_fields": ["id", "url", "body"],
               "filters": [("post_type", "answer")],               
               "limit": 5}
    docs = outdoors_collection.search(**request)["docs"]
    for doc in docs:
        contexts["id"].append(doc["id"])
        contexts["url"].append(doc["url"])
        contexts["question"].append(question)
        contexts["context"].append(doc["body"])
    return pandas.DataFrame(contexts)

In [5]:
example_contexts = retriever('What are minimalist shoes?')
display_contexts(example_contexts)

id,question,context,url
18376,What are minimalist shoes?,"Minimalist shoes or ""barefoot"" shoes are shoes that provide your feet with some form of protection, but get you as close to a barefoot experience as possible. The styles range from simple ultralight sneakers, to thin almost slipper like shoes, all the way down to stick on shoes: image source image source",https://outdoors.stackexchange.com/questions/18376
18370,What are minimalist shoes?,"There was actually a project done on the definition of what a minimalist shoe is and the result was ""Footwear providing minimal interference with the natural movement of the foot due to its high flexibility, low heel to toe drop, weight and stack height, and the absence of motion control and stability devices"". If you are looking for a simpler definition, this is what Wikipedia says, Minimalist shoes are shoes intended to closely approximate barefoot running conditions. 1 They have reduced cushioning, thin soles, and are of lighter weight than other running shoes, allowing for more sensory contact for the foot on the ground while simultaneously providing the feet with some protection from ground hazards and conditions (such as pebbles and dirt). One example of minimalistic shoes would be the Vibram FiveFingers shoes which look like this. Image Source",https://outdoors.stackexchange.com/questions/18370
16427,What are minimalist shoes?,"One summer job, I needed shoes to walk on a rocky beach, sometimes in the water, for 5 to 10 miles per day all summer. Stretchy neoprene shoes were terrible for this- no traction and no support. So I used regular sneakers. I chose a pair of cross country racing flats. The model I used are no longer made, but were made by New Balance. The uppers were extremely well ventilated polyester, so they drained very quickly, and the thin material dried much faster than a padded sandal, and certainly much faster than a regular sneaker of leather or cotton would. There are probably other minimalist sneakers that would be equally good. The thing to look for is thin fabric that attaches directly to the sole, with no rubber rim that would keep the water from draining. Running shoes in general will be good because those tend to be very well ventilated, and minimalist shoes will be best because there will be very little padding to hold extra water.",https://outdoors.stackexchange.com/questions/16427
18375,What are minimalist shoes?,"The answer to this question will vary on your level of foot strength and the terrain. Human beings did pretty well for thousands of years without supportive footwear, but when they started walking on hard stone and climbing rocky mountains they quickly discovered their feet weren't suited for it. You'll notice that most large mammals that live at high elevation have hooves. Most people entering the minimalist barefoot shoe niche grew up wearing sneakers that provided some level of support for your arch/ankle/etc., which has the byproduct of under-developing the stabilizer muscles in your feet as you grow and mature. The result is very sore feet after using the minimalist shoes for long periods of time; the risk is causing a muskuloskeletal disorder from straining these muscles by using them excessively before they have the opportunity to properly develop. It's not unhealthy to wear minimalist footwear, but on what terrain your wear them could be bad for your body in the long run. Human beings were never meant to walk or run exclusively on hard pavement or sidewalks. Nor were we designed to clamber around on sharp rocks at high elevations. Homo Sapiens are a low-elevation tropical animals, You'll notice the vast majority of the world's population is still in Southern Asia, which is very tropical. We had to adapt technology like clothes and footwear in order to thrive in other environments. If you're running on soft ground and you have the foot strength, then there are plenty of arguments in favour of minimalist shoes being better for you th

## Listing 14.16

In [6]:
import torch

def get_processor_device():
    return 0 if torch.cuda.is_available() else -1

In [7]:
from transformers import pipeline

device = get_processor_device()
model_name = "data/question-answering/roberta-base-squad2-fine-tuned"
qa_nlp = pipeline("question-answering", model=model_name,
                  tokenizer=model_name, device=device)

def reader(contexts):
    answers = []
    for _, row in contexts.iterrows():
        answer = qa_nlp({"question": row["question"],
                         "context": row["context"]})
        answer["id"] = row["id"]
        answer["url"] = row["url"]
        answers.append(answer)
    return answers

OSError: Incorrect path_or_model_id: 'data/question-answering/roberta-base-squad2-fine-tuned'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

## Listing 14.17

In [33]:
def reranker(answers):
    return sorted(answers, key=lambda k: k["score"], reverse=True)

## Listing 14.18

In [34]:
import urllib.parse
def print_answer(question, reranked):
    url = "https://outdoors.stackexchange.com/search?q=" + urllib.parse.quote(question)
    html = f'<h2><a href="{url}" target=_blank>{question}</a></h2>'
    for answer in reranked:
         html += f'<h3><a href="{answer["url"]}" target=_blank> {answer["id"]} </a>'
         html += '<strong>' + answer["answer"] + '</strong><em> (' + str(round(answer["score"], 3)) + ')</em></h3>'
    display(HTML(html))

In [35]:
def ask(question):
    documents = retriever(question)
    answers = reader(documents)
    reranked = reranker(answers)
    print_answer(question, reranked)

In [41]:
ask('What is the best mosquito repellent?')
ask('How many miles can a person hike day?')
ask('How much water does a person need per day?')

In [37]:
ask('What material are good walking poles made of?')

In [38]:
ask('How many people can fit inside a 2-person tent?')

Up next: [Chapter 15 - Foundation Models and Emerging Search Paradigms](../ch15/1.llm-exploration.ipynb)